# PyESAPI: Getting Started
We assume `pip install git+https://github.com/VarianAPIs/PyESAPI` has already been executed in your conda3 environment.

## Import PyESAPI
* Create one instance of the eclipse scripting application
* Set the script name for logging purposes to "python_demo"
* Regester the application disposal fumctopm at exit (avoids unexpected exit/crash of app instance)

In [3]:
import pyesapi
import atexit
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

## Find a Patient
* List patient Ids
* Open a patient
* Print patient info
* Python 3.6 f-string sugar

In [4]:
for pat_sum in app.PatientSummaries:
    print(pat_sum.Id)

ref brain
connie breast pt
191001
TestUpperBody
HeartDVHErrorV35Gycc


In [6]:
patient = app.OpenPatientById('TestUpperBody')

In [7]:
print('Name: {patient.FirstName}, LastName: {patient.LastName}'.format(**locals()))

Name: Blueberry, LastName: Pizza


In [21]:
print(f'Name: {patient.FirstName}, LastName: {patient.LastName}')

Name: Blueberry, LastName: Pizza


## A lot about Lots
* Create an array with pythonnet collection iterator
* Demo the PyESAPI Lot
  * Indexing
  * Constructor Shortcuts
  * Constructor Select
  * Lot.FirstOrDefault

In [9]:
[c.Id for c in patient.Courses] # python list comprehension on pythonnet collection iterator

['C1']

In [10]:
patient.CoursesLot() # "Lot" a custom PyESAPI collection wrapper class

In [11]:
patient.CoursesLot()[0].Id # Indexable after construction

'C1'

In [12]:
patient.CoursesLot(0).Id # Indexable at construction

'C1'

In [13]:
patient.CoursesLot('C1').Id # passing string will match Id

'C1'

In [14]:
patient.CoursesLot(lambda c: c.Id == 'C1').Id # passing function acts like "first or default"

'C1'

In [16]:
patient.CoursesLot().FirstOrDefault(lambda c: c.Id == 'C1').Id

'C1'

## Open a Plan
* Print plan info
* Print structure info
* Print beam info

In [17]:
plan = patient.CoursesLot('C1').PlanSetupsLot(0)

In [20]:
print(f'Plan Id: {plan.Id}')
print(f'Dose Per Fx: {plan.PrescribedDosePerFraction}'
print(f'Number of Fx: {plan.NumberOfFractions}')

Plan Id: IMRT, Dose Per Fx: 2000.0 cGy, Number of Fx: 30


In [15]:
print(f'TYPE,ID,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.DicomType},{structure.Id},{structure.Volume:.2f}')

TYPE,ID,VOLUME
PTV,Heart,66.71
EXTERNAL,BODY,9515.75
AVOIDANCE,R Lung,939.14
AVOIDANCE,L Lung,760.37
ORGAN,Cord,39.26
AVOIDANCE,Total Lung,1705.62
PTV,Lg Tumor,825.51


In [16]:
print(f'ID,SSD,Mu,Angle')
for beam in plan.Beams:
    print(f'{beam.Id},{beam.SSD:.2f},{beam.Meterset.Value:.2f},{beam.ControlPoints[0].GantryAngle}')

ID,SSD,Mu,Angle
Field 1,885.74,95.03,315.0
Field 2,867.56,99.98,45.0
Field 3,895.86,99.98,135.0
Field 4,898.13,101.96,225.0


In [17]:
ct_image = plan.StructureSet.Image.np_array_like() # an actual numpy array

In [18]:
type(ct_image)

numpy.ndarray

In [19]:
ct_image.shape # indexed like x, y, z

(512, 512, 237)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(ct_image[:,:,110].T, cmap='gray')
plt.show()

In [ ]:
pyESAPI.SAFE_MODE = False
dose = plan.Dose.np_array_like(plan.StructureSet.Image) # create dose at CT resolution

In [ ]:
plt.imshow(dose[:,:,110].T,cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
structure = plan.StructureSet.StructuresLot('Heart').np_mask_like(plan.StructureSet.Image)

In [ ]:
import numpy as np
dose_in_structure = dose*structure
plt.imshow(dose_in_structure[:,:,110].T, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
non_zero_dose_in_structure = np.ma.masked_where(dose_in_structure==0,dose_in_structure)
plt.imshow(non_zero_dose_in_structure[:,:,110].T, cmap='jet')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(ct_image[100:400,175:400,110].T, cmap='gray')
plt.imshow(non_zero_dose_in_structure[100:400,175:400,110].T, cmap='jet',alpha=.25)
# plt.colorbar()
plt.show()

In [ ]:
# challenge: overlay all dose above 2 Gy

In [ ]:
# add interactivity
from ipywidgets import interactive, widgets

def plot_function(slice_num):
    plt.figure(figsize=(8,8))
    plt.imshow(ct_image[100:400,175:400,slice_num].T, cmap='gray')
    plt.plot()

islide = widgets.IntSlider(
    value=110,
    min=0,
    max=200,
    step=1,
    description='Slice Number',
    continuous_update=True,
)
    
interactive(plot_function, slice_num=islide)